In [1]:
# Dependencies and Setup
import os
import csv
import math
import statistics
import pandas as pd
import numpy as np

In [2]:
# File to Load
school_data_row = "Resources/schools_complete.csv"
student_data_row = "Resources/students_complete.csv"
print(f"Both files were found")

Both files were found


In [3]:
# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_row)

# Read School and Student Data File and store into Pandas Data Frames
student_df = pd.read_csv(student_data_row)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

# See as a DataFrame and check it out
data_df = pd.DataFrame(school_data_complete)
data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
# Looking for the ones that pass math
pass_math_df = data_df[data_df["math_score"]>=70]
pass_math_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635


In [5]:
pass_math_df["Student ID"].nunique()

29370

In [6]:
percentage_pass_math = (pass_math_df["Student ID"].nunique() / data_df["Student ID"].nunique())*100
percentage_pass_math

74.9808526933878

In [7]:
# Looking for the ones that pass reading
pass_reading_df = data_df[data_df["reading_score"]>=70]
pass_reading_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635


In [8]:
pass_reading_df["Student ID"].nunique()

33610

In [9]:
percentage_pass_reading = (pass_reading_df["Student ID"].nunique() / data_df["Student ID"].nunique())*100
percentage_pass_reading

85.80546336482001

In [10]:
percentage_overall_passing = (percentage_pass_math + percentage_pass_reading)/2
percentage_overall_passing

80.39315802910392

In [11]:
school_df["budget"].sum()

24649428

In [12]:
# Creating a summary DataFrame using the values found
district_summary_df = pd.DataFrame({
    "Total Schools":[data_df["school_name"].nunique()],
    "Total Students" : [data_df["Student ID"].nunique()],
    "Total Budget" : [school_df["budget"].sum()],
    "Average Math Score" : [data_df["math_score"].mean()],
    "Average Reading Score" : [data_df["reading_score"].mean()],
    "% Passing Math":[percentage_pass_math],
    "% Passing Reading": [percentage_pass_reading],
    "% Overall Passing Rate":[percentage_overall_passing]
})

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [13]:
# Checking the kind of variables
data_df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [18]:
# Using GroupBy in order to separate the data into fields according to "school" values
grouped_schools_df = data_df.groupby(["school_name"])

# In order to be visualized, a data function must be used
#grouped_schools_df.count().head(15)

In [19]:
# The numeric variables can be now calculated per school 
# Total Students
total_students = grouped_schools_df["Student ID"].count()
total_students.head(15)

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: Student ID, dtype: int64

In [20]:
# The numeric variables can be now calculated per school 
# Average Math Score
school_average_math = grouped_schools_df["math_score"].mean()
school_average_math.head(15)

school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

In [21]:
# The numeric variables can be now calculated per school 
# Average Reading Score
school_average_reading = grouped_schools_df["reading_score"].mean()
school_average_reading.head(15)

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [ ]:
# VERIFY IF PRINTING RESULTS IS GOING TO WORK -- Creating a SCHOOL summary DataFrame using the values found
#"% Passing Math":[perc_pass_math_value["% Passing Math"].],
#    "% Passing Reading": [percentage_pass_reading],
#    "% Overall Passing Rate":[percentage_overall_passing]
#"School Name":[grouped_schools_df["school_name"]],
#    "Total Students" : [grouped_schools_df["Student ID"].count()],

school_summary_df = pd.DataFrame({
    "Average Math Score" : [school_average_math],
    "Average Reading Score" : [school_average_reading]
})

school_summary_df.head()

In [ ]:
# BEFORE HERE Looking for the ones that pass math per school
pass_math_per_school = pass_math_df.groupby(["school_name"])
pass_math_per_school.count().head(15)

In [ ]:
# In order to get the percentage passing math
perc_pass_math = (pass_math_per_school.count() / grouped_schools_df.count())
perc_pass_math_value = perc_pass_math[["Student ID"]]*100 
perc_pass_math_value = perc_pass_math_value.rename(columns={"Student ID":"% Passing Math"})
perc_pass_math_value.head(15)

In [ ]:
# Looking for the ones that pass reading per school
pass_reading_per_school = pass_reading_df.groupby(["school_name"])
pass_reading_per_school.count().head(15)

In [ ]:
# In order to get the percentage passing reading
perc_pass_reading = (pass_reading_per_school.count() / grouped_schools_df.count())
perc_pass_reading_value = perc_pass_reading[["Student ID"]]*100 
perc_pass_reading_value = perc_pass_reading_value.rename(columns={"Student ID":"% Passing Reading"})
perc_pass_reading_value.head(15)

In [ ]:
# What kind of variables are the percentages I got?
perc_pass_math_value.dtypes

In [ ]:
# What kind of variables are the percentages I got?
perc_pass_reading_value.dtypes

In [ ]:
# The numeric variables can be now calculated per school 
# Average Math Score
grouped_schools_df["school_average_math"] = grouped_schools_df["math_score"].mean(axis=1)
grouped_schools_df.head(15)

In [ ]:
# Calculate the overall Passing rate per school
# Place the results into a new column
summary_passing_per_school = pd.merge([perc_pass_math_value["% Passing Math"]], 
                                      [perc_pass_reading_value["% Passing Reading"]]
                                     )
summary_passing_per_school.head(15)

In [ ]:
# Set new index to school name
school_df = data_df.set_index("school_name")
school_df.head()

In [ ]:
# School Summary
# Create an overview table that summarizes key metrics about each school, including:
# School Name
# School Type
# Total Students
# Total School Budget
# Per Student Budget
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)